# Goal : Implement Locality Sensitive Hashing (LSH) to perform ANN Similarity Search

- At it's core, LSH tries to reduces our search space by running vectors through a variety of hash functions and grouping vectors with similar hashes into same buckets.
- Now if we have a search query vector, we only look in the buckets that have a similarity match to the query vector.
- The concept is analogous to how dictionaries/hashmaps try to hash a given key. In general, these hash functions are such that they avoid collisions of keys(even though the keys are close to similar).In case of LSH hash functions, these functions try to increase collisions for vectors that might be similar so that they can grouped in the same bucket.
- There are two widely used techniques:
    - Traditional Technique : Uses shingling -> MinHashing -> LSH function
    - Random Projection: Random Hyperplanes with DotProduct divides data points into multiple subsets -> Hamming Distance

### Traditional Technique

#### Shingling
- k-Shingling or Shingling - is the process of converting a string of text into a set of 'shingles'.
- In this process to create shingle, we move a window of size 'k' across the text. At each point we take a snapshot of the text to create one shingle
- We place all these shingles into a set data structure to remove any duplicates.

In [33]:
# let's take 3 sample strings to test

a = "A group of kids is playing in a yard and an old man is standing in the background"
b = "The young boys are playing outdoors and the man is smiling nearby"
c = "Two children are lying in the snow and are drawing angels"

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 38, Finished, Available)

In [34]:
# given a string, generate k window sized shingles and return unique shingles

def gen_shingles(text: str, k: int=2):
    shingles = [text[i:i+k] for i in range(len(text)-k)]
    return set(shingles)

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 39, Finished, Available)

In [35]:
a_shingles = gen_shingles(a)
b_shingles = gen_shingles(b)
c_shingles = gen_shingles(c)

print(a_shingles)

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 40, Finished, Available)

{'d ', 'ay', ' y', 'th', 'he', 'ba', 'un', 'ta', ' k', 'ac', 'pl', 'of', 'gr', 'st', ' b', ' g', 'ma', ' m', 'ck', 'ds', 'la', 'ou', 'p ', 'an', ' p', 'f ', 'g ', 'nd', 'kg', 's ', 'a ', 'A ', ' s', 'di', 'ya', 'ld', 'in', ' a', ' i', 'ki', 'yi', 'ro', 'n ', 'ar', ' o', ' t', 'up', 'is', 'rd', 'ng', 'id', 'ol', 'e '}


#### Create Sparse Vectors to represent these shingles
- First we will create a vocab set which is set of all the shingles in our test vocabulary
- One Hot encode our shingles against this vocab set

In [36]:
# create a vocab set 

vocab = a_shingles.union(b_shingles).union(c_shingles)

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 41, Finished, Available)

In [37]:
# declare 3 lists for holding 1 hot encoding for a, b, c shingles
a_1hot = []
b_1hot = []
c_1hot = []

# iterate through elements in vocab and create 1 hot encoded vectors for a, b, c
for v in vocab:
    a_1hot.append(1 if v in a_shingles else 0)
    b_1hot.append(1 if v in b_shingles else 0)
    c_1hot.append(1 if v in c_shingles else 0)

print(a_1hot)

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 42, Finished, Available)

[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]


#### Minhashing

- Allows us convert these sparse vectors to dense vectors
- If we wanted to create a dense vector of size 30, we will use 30 minhash functions (one hash function for every position in the dense vector)
- Minhash function is nothing but a random order of numbers between 1 and len(vocab) and then find the minimum number that aligns with a 1 in the sparse vector.
- Since we wanted to produce 30 (or more) of these values to produce a signature (our dense vector), we need to repeat step 5 30 (or more) times to generate the signature.

In [38]:
from random import shuffle

# generate a random order of numbers our first minhash_vec
minhash_vec = list(range(1, len(vocab)+1))
shuffle(minhash_vec)
# print(minhash_vec)

# let's find the smallest number, by iterating
for i in range(1, len(vocab)+1):
    # find index of i in minhash_vec
    idx = minhash_vec.index(i)

    print(f"iteration: {i} -> index of {i} in minhash_vec: {idx} -> value at {idx} in sparse vector : {a_1hot[idx]} \n")

    # does the sparse vector has '1' at this index?
    if a_1hot[idx] == 1:
        print(f'add {idx} to the signature')
        break;




StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 43, Finished, Available)

iteration: 1 -> index of 1 in minhash_vec: 77 -> value at 77 in sparse vector : 1 

add 77 to the signature


#### Now that we saw how minhashing works, Let's create few functions to run through our samples

In [39]:
# create the hash func that provides us the random order of numbers

def create_hash_func(size: int):
    hash_vec = list(range(1, size+1))
    shuffle(hash_vec)
    return hash_vec


# create multiple such hashes one for each position in dense vector

def build_minhash_function(vocab_size:int, dense_vector_size:int):
    hash_vecs = []
    for _ in range(dense_vector_size):
        hash_vecs.append(create_hash_func(vocab_size))

    return hash_vecs


minhash_func = build_minhash_function(len(vocab), 50)

# create signature or dense vectors for a given sparse vector using minhash function
def create_minhash_signature(sparse_vec: list):
    vocab_size = len(vocab)
    signature = []

    # iterate through each hash vector
    for hash_vec in minhash_func:
        for i in range(1, vocab_size+1):
            idx = hash_vec.index(i)
            if sparse_vec[idx] == 1:
                signature.append(idx)
                break;
    
    return signature

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 44, Finished, Available)

In [40]:
# let's create minhash dense vectors for a_1hot, b_1hot and c_1hot sparse vectors

a_sig = create_minhash_signature(a_1hot)
b_sig = create_minhash_signature(b_1hot)
c_sig = create_minhash_signature(c_1hot)

print(c_sig)

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 45, Finished, Available)

[71, 51, 71, 62, 29, 23, 29, 46, 29, 42, 47, 15, 31, 90, 32, 15, 73, 15, 20, 70, 23, 45, 70, 83, 84, 82, 30, 78, 28, 21, 1, 57, 82, 83, 71, 23, 83, 30, 83, 78, 67, 28, 81, 71, 75, 47, 80, 20, 73, 70]


Since we compressed down a sparse vector to a dense vector of size 30, let's compare the similarities using Jaccard distance to see if any info is lost due to this compression.

In [42]:
def jaccard_distance(x: set, y: set):    
    return len(x.intersection(y))/len(x.union(y))

print(f'Jaccard Distance for Original A and B {jaccard_distance(a_shingles, b_shingles)}')
print(f'Jaccard Distance for Dense Signatures of A and B {jaccard_distance(set(a_sig), set(b_sig))}')
print('\n')
print(f'Jaccard Distance for Original A and C {jaccard_distance(a_shingles, c_shingles)}')
print(f'Jaccard Distance for Dense Signatures of A and C {jaccard_distance(set(a_sig), set(c_sig))}')
print('\n')
print(f'Jaccard Distance for Original B and C {jaccard_distance(b_shingles, c_shingles)}')
print(f'Jaccard Distance for Dense Signatures of B and C {jaccard_distance(set(b_sig), set(c_sig))}')

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 47, Finished, Available)

Jaccard Distance for Original A and B 0.3972602739726027
Jaccard Distance for Dense Signatures of A and B 0.28888888888888886


Jaccard Distance for Original A and C 0.22666666666666666
Jaccard Distance for Dense Signatures of A and C 0.18


Jaccard Distance for Original B and C 0.2571428571428571
Jaccard Distance for Dense Signatures of B and C 0.2391304347826087


#### Band and Hash
- Split our signatures into bands.
- Hash each band , if there is a collision place the entire vector in the same bucket
- Why do we band the signatures ? Even if there is a part of the signature that has similarity with other vectors, we need to bucket them together. This will enable us to evaluate candidate pairs that have at least a sense of similarity instead of exact match.
- Obviously this does increase the nbr of false positives, but tuning the band size should help us reduce them.
- Since we have a 50 dimensional signature, if we split it into bands of size 10. It gives us the opportunity to check 5 such matches of sub-vectors.
- ![Band and Hash](./resources/BandNHash.png)

In [43]:
# Let's split the vector into sub vectors
def create_sub_vectors(signature_vec:list, subvec_size:int):
    # we want the subvec_size to split equal subvectors with out leaving any data behind
    assert(len(signature_vec)%subvec_size == 0)
    sub_vectors = []
    for i in range(0, len(signature_vec), subvec_size):
        sub_vectors.append(signature_vec[i:i+subvec_size])

    return sub_vectors

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 48, Finished, Available)

In [44]:
sig_bands_a = create_sub_vectors(a_sig, 5)
sig_bands_b = create_sub_vectors(b_sig, 5)
sig_bands_c = create_sub_vectors(c_sig, 5)

print(f"subvec bands for a {sig_bands_a}")
print(f"subvec bands for b {sig_bands_b}")
print(f"subvec bands for c {sig_bands_c}")

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 49, Finished, Available)

subvec bands for a [[6, 5, 71, 12, 29], [34, 63, 46, 29, 13], [56, 16, 27, 19, 6], [16, 21, 77, 20, 6], [29, 2, 19, 17, 5], [82, 16, 49, 28, 19], [64, 19, 53, 12, 71], [34, 12, 53, 18, 7], [67, 28, 10, 2, 69], [27, 80, 27, 49, 48]]
subvec bands for b [[37, 66, 13, 58, 29], [34, 29, 46, 29, 13], [66, 15, 14, 19, 37], [15, 21, 50, 20, 66], [29, 2, 19, 37, 84], [82, 4, 49, 28, 19], [74, 37, 66, 66, 26], [54, 31, 37, 50, 19], [67, 28, 34, 2, 8], [22, 80, 74, 49, 9]]
subvec bands for c [[71, 51, 71, 62, 29], [23, 29, 46, 29, 42], [47, 15, 31, 90, 32], [15, 73, 15, 20, 70], [23, 45, 70, 83, 84], [82, 30, 78, 28, 21], [1, 57, 82, 83, 71], [23, 83, 30, 83, 78], [67, 28, 81, 71, 75], [47, 80, 20, 73, 70]]


In [45]:
from itertools import combinations

# Let's maintain buckets for each band (we have 10 bands)
buckets = [{} for _ in range(10)]

# take the list of bands, vec_indicator indicates the vector bands belong to
def hash_sig_bands(bands: list, vec_indicator: int):
    for i, band in enumerate(bands):
        # perform a simple string join and use default dict hash in python
        simple_hash = ','.join([str(i) for i in band])

        # if the hash key is not present, create an empty list
        if simple_hash not in buckets[i]:
            buckets[i][simple_hash] = []

        # append the vector indicator for this key
        buckets[i][simple_hash].append(vec_indicator)


def get_candidate_pairs():
    candidates = []

    # iterate through all buckets
    for bucket in buckets:
        # look for keys with multiple vector indicators (indicating a collision/hash match)
        for key in bucket.keys():
            if len(bucket[key]) > 1:
                # add all possible vector pair combinations to candidates
                candidates.extend(combinations(bucket[key], 2))
    
    # remove any duplicates
    return set(candidates)


StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 50, Finished, Available)

In [46]:
# hash all the signature bands

hash_sig_bands(sig_bands_a, 1)
hash_sig_bands(sig_bands_b, 2)
hash_sig_bands(sig_bands_c, 3)

# display candidate pairs that are similar
get_candidate_pairs()

StatementMeta(15d5176f-8c31-4590-b30b-49036d1913db, 5, 51, Finished, Available)

set()

## Random Projection for LSH

- ** Reduce the Dimensionality: ** We reduce the dimensionality of the sparse vector by using hyperplanes. 
    - Each hyperplane will divide the space into two halves.
    - Any point on the +ve side of the hyperplane, the resulting dense vector is assigned a '1'
    - Any point on the -ve side of the hyperplane, the resulting dense vector is assigned a '0'

- ** How do we identify, which side the point is on: **
    - Take a normal vector (any vector perpendicular to the plane)
    - Perform a dotproduct between the normal vector and the data point
    - If the normal vector and the data point are in the same direction , then the dot product is positive
    - If the normal vector and the data point are in the opposite direction, then the dot product is negative.
    - If the point is sitting on the edge of the hyperplane , then the dot product is zero. We can group it along with the negative dot product.    

- ** Create Hashed Vectors: **
    - Introduce multiple such Hyperplanes, if we want dense vector of 'n' dimensions. We will introduce 'n' such hyperplanes and then perform dotproduct to create an 'n' dimensional dense vector.
    - Hash the dense vector and place all the vectors with similar hashes into buckets.

- ** Search Complexity: **
    - Now that we have bucketed all our vectors, let's consider how the search complexity is reduced.
    - Given a query vector, gets converted to dense vector and then gets hashed following similar steps
    - Hashed query vector will be compared against just the candidates in the bucket that match it's hash, thus reducing the search complexity from linear to sub-linear.
    - With in the matching bucket, we compare the query vector with the candidates in the matching bucket. Then identify the closest matching candidates using Hamming distance.
    